In [54]:
import numpy as np
import pandas as pd
from statsmodels.sandbox.regression.gmm import GMM


# Part 1

In [69]:
df = pd.read_csv('https://raw.githubusercontent.com/EvaW01/schulich_data_science_Xinyue-Wang/refs/heads/main/small_retailers_stock_performance.csv')

In [70]:
df

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
0,1,Low,No dependent,Very low,Very low,Married,Positive
1,2,Low,No dependent,Very low,Very low,Single,Positive
2,1,Low,No dependent,Very low,Very low,Single,Positive
3,3,Low,No dependent,Very low,Very low,Married,Positive
4,3,Low,No dependent,Very low,Very low,Single,Negative
...,...,...,...,...,...,...,...
8076,3,Low,Less than 2,Very High,Very high,Married,Positive
8077,3,Medium,Less than 2,Very High,Very high,Married,Negative
8078,3,Medium,More than 2,Very High,Very high,Married,Positive
8079,7,Medium,Less than 2,Very High,Very high,Married,Positive


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8081 entries, 0 to 8080
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Years of Education after High School  8081 non-null   int64 
 1   Requested Credit Amount               8081 non-null   object
 2   Number of Dependents                  8081 non-null   object
 3   Monthly Income                        8081 non-null   object
 4   Monthly Expense                       8081 non-null   object
 5   Marital Status                        8081 non-null   object
 6   Credit Rating                         8081 non-null   object
dtypes: int64(1), object(6)
memory usage: 442.1+ KB


In [58]:
# Assume that y_vals, x_vals, iv_vals are the dependent variable, independent variable and instrumental variable respectively
y_vals = np.array(df["Stock Change"])
x_vals = np.array(df[["Inventory Turnover", "Operating Profit", "Interaction Effect"]])
iv_vals = np.array(df[["Current Ratio", "Quick Ratio", "Debt Asset Ratio"]])

In [65]:
class gmm(GMM):
    def momcond(self, params):
        # Add δ to the parameters
        p0, p1, p2, p3, delta = params

        endog = self.endog
        exog = self.exog
        inst = self.instrument   

        # Raw error term
        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]

        
        # Add δ to the moment conditions associated with the instrumental variable
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0] + delta
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] + delta
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] + delta

        # Construct moment condition matrix
        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g

# Set initial parameters, including δ
beta0 = np.array([0.1, 0.1, 0.1, 0.1, 0.1])

# Create and fit the model
res = gmm(endog=y_vals, exog=x_vals, instrument=iv_vals, k_moms=6, k_params=5).fit(beta0)
res.summary()


Optimization terminated successfully.
         Current function value: 0.000031
         Iterations: 10
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: 0.000345
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 7
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.5862
Model:                            gmm   Prob (Hansen J):                 0.444
Method:                           GMM                                         
Date:                Sat, 09 Nov 2024                                         
Time:                        21:55:16                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0208      0.021     -0.986      0.324      -0.062       0.020
p 1            0.0011      0.001      1.839      0.066   -7.31e-05       0.002
p 2           -0.1062      0.032     -3.316      0.001      -0.169      -0.043
p 3            0.0011      0.000      2.688      0.007       0.000       0.002
p 4            0.0006      0.003      0.213      0.831      -0.005       0.006
==============================================================================
"""

### 优化过程结果

优化信息显示模型成功收敛，最终函数值较小，表示拟合程度较好：

- **迭代次数**（Iterations）：多次优化过程表明模型通过 10 到 2 次迭代成功收敛。
- **函数评估与梯度评估次数**（Function evaluations & Gradient evaluations）：这些值显示优化过程的计算量，表明模型在多次评估后收敛。

### Hansen J 统计量

- **Hansen J 统计量**（Hansen J）：0.5862。
- **p 值**（Prob (Hansen J)）：0.444。
  - Hansen J 统计量用于检验过度识别限制（即工具变量的有效性）。p 值 0.444 大于 0.05，表明没有足够证据拒绝工具变量的有效性假设。因此，工具变量是有效的，符合模型假设。

### 系数估计和显著性

| 参数 | 系数估计 | 标准误差 | z 值 | p 值 | 95% 置信区间 |
|------|----------|----------|------|------|--------------|
| p0   | -0.0208  | 0.021    | -0.986 | 0.324 | [-0.062, 0.020] |
| p1   | 0.0011   | 0.001    | 1.839  | 0.066 | [-0.0000731, 0.002] |
| p2   | -0.1062  | 0.032    | -3.316 | 0.001 | [-0.169, -0.043] |
| p3   | 0.0011   | 0.000    | 2.688  | 0.007 | [0.000, 0.002] |
| p4   | 0.0006   | 0.003    | 0.213  | 0.831 | [-0.005, 0.006] |

- **p0、p1、p2、p3**：这些系数分别对应不同自变量的影响。
  - **p2** 和 **p3** 的 p 值小于 0.05，显著不同于零，表明这些变量对因变量有显著影响。
  - **p1** 的 p 值为 0.066，接近 0.05 水平，但不完全显著。
  - **p0** 的 p 值为 0.324，不显著。

- **p4**（偏差项 \( \delta \)）：
  - 估计值为 0.0006，标准误差为 0.003。
  - **p 值**：0.831，远高于 0.05 显著性水平，表明偏差项 \( \delta \) 不显著。
  - **解释**：由于 \( \delta \) 的 p 值较高，无法拒绝 \( \delta = 0 \) 的假设，因此没有足够证据表明在 moment 条件中存在系统性偏差。这说明行业专家的偏差假设不成立。

### 结论

- **工具变量的有效性**：Hansen J 检验表明工具变量是有效的。
- **偏差项的显著性**：偏差项 \( \delta \) 不显著，表明 moment 条件中没有系统性偏差。这与行业专家的主张不一致，表明行业专家的偏差假设不成立。
- **其他系数的显著性**：一些系数（如 `p2` 和 `p3`）对因变量有显著影响，但偏差项 \( \delta \) 并未显示出显著性。

### Optimization Results

The optimization process indicates successful convergence, with a low final function value, suggesting a good model fit:

- **Iterations**: The model converged successfully through multiple optimization processes, taking between 2 and 10 iterations.
- **Function and Gradient Evaluations**: These values show the computational steps involved, with the model converging after multiple evaluations of the function and its gradient.

### Hansen J Statistic

- **Hansen J Statistic**: 0.5862
- **p-value (Hansen J)**: 0.444
  - The Hansen J statistic tests the validity of the over-identifying restrictions, which essentially means testing the validity of the instrumental variables. A p-value of 0.444, which is greater than 0.05, indicates that we do not have enough evidence to reject the null hypothesis that the instruments are valid. This suggests that the instrumental variables are appropriate and consistent with the assumptions of the GMM model.

### Coefficient Estimates and Significance

| Parameter | Coefficient | Std. Error | z-Score | p-Value | 95% Confidence Interval |
|-----------|-------------|------------|---------|---------|--------------------------|
| p0        | -0.0208     | 0.021      | -0.986  | 0.324   | [-0.062, 0.020]          |
| p1        | 0.0011      | 0.001      | 1.839   | 0.066   | [-0.0000731, 0.002]      |
| p2        | -0.1062     | 0.032      | -3.316  | 0.001   | [-0.169, -0.043]         |
| p3        | 0.0011      | 0.000      | 2.688   | 0.007   | [0.000, 0.002]           |
| p4        | 0.0006      | 0.003      | 0.213   | 0.831   | [-0.005, 0.006]          |

- **p0, p1, p2, p3**: These coefficients correspond to the different explanatory variables in the model.
  - **p2** and **p3** have p-values less than 0.05, indicating statistical significance at the 5% level. This suggests that these variables have a significant impact on the dependent variable.
  - **p1** has a p-value of 0.066, which is close to the 0.05 significance level but is not strictly significant.
  - **p0** has a p-value of 0.324, which is not significant, suggesting that this variable does not have a meaningful impact on the dependent variable.

- **p4** (the bias term \( \delta \)):
  - The estimated value for \( \delta \) is 0.0006, with a standard error of 0.003.
  - **p-Value**: The p-value for \( \delta \) is 0.831, which is much higher than the usual significance threshold (e.g., 0.05).
  - **Interpretation**: A high p-value for \( \delta \) means that we do not have sufficient evidence to reject the null hypothesis that \( \delta = 0 \). In other words, there is no statistically significant evidence of a systematic bias in the moment conditions. This result does not support the industry expert’s claim of a non-zero bias term \( \delta \).

### Conclusion

- **Instrument Validity**: The Hansen J test suggests that the instrumental variables are valid, as we cannot reject the null hypothesis of the test.
- **Significance of the Bias Term**: The bias term \( \delta \) is not statistically significant, indicating that there is no systematic bias in the moment conditions. This result is inconsistent with the industry expert’s claim, suggesting that the expert’s hypothesis of a non-zero bias is not supported by the data.
- **Significance of Other Coefficients**: Some coefficients (such as `p2` and `p3`) show a significant effect on the dependent variable, while the bias term \( \delta \) does not.

In summary, the GMM results do not provide statistical evidence to support the industry expert's claim that there is a non-zero bias in the moment conditions. The high p-value for \( \delta \) suggests that the bias term is effectively zero, indicating that the moment conditions remain unbiased.

# Part 2

### 1.

In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [72]:
df_2 = pd.read_csv('https://raw.githubusercontent.com/EvaW01/schulich_data_science_Xinyue-Wang/refs/heads/main/midterm_parttwo.csv')

In [73]:
df_2

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
0,1,Low,No dependent,Very low,Very low,Married,Positive
1,2,Low,No dependent,Very low,Very low,Single,Positive
2,1,Low,No dependent,Very low,Very low,Single,Positive
3,3,Low,No dependent,Very low,Very low,Married,Positive
4,3,Low,No dependent,Very low,Very low,Single,Negative
...,...,...,...,...,...,...,...
8076,3,Low,Less than 2,Very High,Very high,Married,Positive
8077,3,Medium,Less than 2,Very High,Very high,Married,Negative
8078,3,Medium,More than 2,Very High,Very high,Married,Positive
8079,7,Medium,Less than 2,Very High,Very high,Married,Positive


In [74]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8081 entries, 0 to 8080
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Years of Education after High School  8081 non-null   int64 
 1   Requested Credit Amount               8081 non-null   object
 2   Number of Dependents                  8081 non-null   object
 3   Monthly Income                        8081 non-null   object
 4   Monthly Expense                       8081 non-null   object
 5   Marital Status                        8081 non-null   object
 6   Credit Rating                         8081 non-null   object
dtypes: int64(1), object(6)
memory usage: 442.1+ KB


In [82]:
# Make a copy of df_2 for preprocessing
df_encoded = df_2.copy()

# Encoding all categorical variables using LabelEncoder for simplicity
for column in df_encoded.columns:
    if df_encoded[column].dtype == 'object':
        le = LabelEncoder()
        df_encoded[column] = le.fit_transform(df_encoded[column])

In [84]:
# Splitting data into features (X) and target (y)
X = df_encoded.drop('Credit Rating', axis=1)
y = df_encoded['Credit Rating']

# Splitting data into 50% training and 50% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [85]:
# Training logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [86]:
# Predicting on the test set
y_pred = model.predict(X_test)

In [90]:
# Calculating each metric separately
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

array([[   0,  577],
       [   0, 3464]])

In [91]:
recall = recall_score(y_test, y_pred, pos_label=1)
recall

1.0

In [89]:
precision = precision_score(y_test, y_pred, pos_label=1)
precision

0.8572135609997525

In [88]:
f1 = f1_score(y_test, y_pred, pos_label=1)
f1

0.9231179213857429

### 2.

In [92]:
# Predict probabilities on the test set
y_prob = model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate threshold to approve only 15% of applications
threshold = sorted(y_prob, reverse=True)[int(len(y_prob) * 0.15)]

# Apply the new threshold
y_pred_thresholded = (y_prob >= threshold).astype(int)

In [96]:
# Calculate updated confusion matrix, recall, precision, and F1 score
conf_matrix_updated = confusion_matrix(y_test, y_pred_thresholded)
conf_matrix_updated

array([[ 506,   71],
       [2928,  536]])

In [95]:
recall_updated = recall_score(y_test, y_pred_thresholded, pos_label=1)
recall_updated

0.15473441108545036

In [94]:
precision_updated = precision_score(y_test, y_pred_thresholded, pos_label=1)
precision_updated

0.8830313014827018

In [93]:
f1_updated = f1_score(y_test, y_pred_thresholded, pos_label=1)

f1_updated, threshold

(0.2633259641365758, 0.8785917263593007)